In [85]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

model = OllamaLLM(model="gemma3:4b")

In [86]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回復客戶郵件。請用友善和專業的語調回覆。請不要回覆(公司的資訊和結尾詞)，我會自行手動增加。"),
    ("human","請回覆這封客戶郵件:\n\n{email_content}")
])

In [87]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    current_time = datetime.datetime.now()
    date_str = current_time.strftime("{y}%Y-%m-%d %H:%M").format(
        y='中華民國,西元'  # 用來替換 {y}
    )
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
治理客服團隊

回覆時間:{date_str}
"""
    return formatted_reply

format_email = RunnableLambda(format_email_reply)

In [88]:
chain = (prompt_template
        | model
        | StrOutputParser()
        | format_email
        )
        

In [89]:
customer_email = """您好:
我最近購買了櫃公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎?我已經購買一週了。

期待您的回覆!

王小明"""

In [90]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)

您好王小明，

非常感謝您聯繫我們，並告知您遇到的問題。 我們深感抱歉您收到的產品有損壞及功能不符的情況，這讓您產生不愉快的購物體驗，我們感到非常抱歉。

請您放心，我們會盡全力協助您處理這個問題。

請您提供以下資訊，以便我們更快速地處理您的退貨或換貨申請：

*   **您的訂單編號**
*   **產品的型號或名稱**
*   **損壞的具體描述**
*   **功能不符的具體描述**
*   **產品照片 (請盡可能清晰地拍攝損壞部位和產品異常情況)**

我們將在收到您的回覆後，立即為您安排最合適的解決方案。 再次感謝您的耐心配合與理解。

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
治理客服團隊

回覆時間:中華民國,西元2025-10-18 15:31

